# Ejercicios Spark SQL

### Ejercicio 1.1 Red social de amigos

En este ejercicio repasaremos los conceptos básicos de Spark usando un conjunto de datos relativos a una red social de amigos. El archivo tiene una estructura como la siguiente, puedes abrirlo para echarle un vistazo con un editor de texto (ej. Bloc de notas, Notepad ++)

| id                     | nombre    | edad             | numero de amigos |
|------------------------|-----------|------------------|------------------|
| ...                    | ...       | ...              | ...              |

1. Copia el archivo amigosRedSocial.csv en una carpeta RedSocial en la carpeta donde estés subidos los datos

2. Crea un Data Frame con PySpark usando el método read para formato csv y usando el esquema custom proporcionado.

3. Realiza las consultas que se indican en celdas posteriores usando funciones de la libreria de Data Frame (SQL Like)

In [6]:
#Importa las librerias necesarias y crear la sesión de Spark
import findspark

findspark.init()

In [7]:
from pyspark.sql import *

spark: SparkSession = SparkSession.builder \
    .appName('IntroSparkSQLAlumno') \
    .config('spark.sql.repl.eagerEval.enabled', True) \
    .config('spark.sql.repl.eagerEval.maxNumRows', 10) \
    .getOrCreate()

In [47]:
#sustituye tu usuario o lo que corresponda de la ruta siguiente según hayas cargado el archivo
path = "file:////home/training/Datos/RedSocial/"

El archivo origen no tiene cabecera ni incluye metadatos sobre su esquema. En estos casos si no indicamos esquema las columnas se crearán con los nombre _c0, _c1, _c2,... Igualmente los tipos de datos se crearan por defecto a String , o bien se tratará de inferir el tipo en el caso de usar la opción de inferir esquema.

- Prueba a leer el archivo y crear una variable denominada peopleSinEsquema para el nuevo Data Frame. Pista: spark.read.format("csv").load(path)

In [48]:
peopleSinEsquema = spark.read.format('csv').option("header", "true").load(path + "amigosRedSocial.csv")

In [49]:
peopleSinEsquema

0,Will,33,385
1,Jean-Luc,26,2
2,Hugh,55,221
3,Deanna,40,465
4,Quark,68,21
5,Weyoun,59,318
6,Gowron,37,220
7,Will,54,307
8,Jadzia,38,380
9,Hugh,27,181
10,Odo,53,191


Imprime el esquema del Data Frame para comprobarlo

In [50]:
peopleSinEsquema.printSchema()

root
 |-- 0: string (nullable = true)
 |-- Will: string (nullable = true)
 |-- 33: string (nullable = true)
 |-- 385: string (nullable = true)



Las columnas y tipos de datos se podrían renombrar y cambiar de tipo con las funciones Data Frame. Sin embargo, una forma más cómoda de tratar estos archivos es definir un esquema previamente a su carga.

In [51]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

customSchema = StructType([ \
  StructField("ID", IntegerType(), True), \
  StructField("name", StringType(), True), \
  StructField("age", IntegerType(), True), \
  StructField("numFriends", IntegerType(), True) \
                          ])

Podemos aplicar el esquema anterior usando la opción del método read: 
**.schema(customSchema)**

In [52]:
people = spark.read.format("csv") \
      .schema(customSchema) \
      .load(path)

In [53]:
people

ID,name,age,numFriends
0,Will,33,385
1,Jean-Luc,26,2
2,Hugh,55,221
3,Deanna,40,465
4,Quark,68,21
5,Weyoun,59,318
6,Gowron,37,220
7,Will,54,307
8,Jadzia,38,380
9,Hugh,27,181


- Imprimir el esquema del DF obtenido en el punto anterior.

In [54]:
people.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- numFriends: integer (nullable = true)



- Selecionar los nombres de las personas y su edad

In [55]:
people.select('name', 'age')

name,age
Will,33
Jean-Luc,26
Hugh,55
Deanna,40
Quark,68
Weyoun,59
Gowron,37
Will,54
Jadzia,38
Hugh,27


- Obtener los datos de personas entre 12 y 20 años utilizando operaciones con DataFrames.

In [56]:
people.select("*").filter(people['age'] >= 12).filter(people['age'] <= 20)
#people.filter(people['age'] >= 12) & (people['age'] <= 20))

ID,name,age,numFriends
21,Miles,19,268
48,Nog,20,1
52,Beverly,19,269
54,Brunt,19,5
60,Geordi,20,100
73,Brunt,20,384
106,Beverly,18,499
115,Dukat,18,397
133,Quark,19,265
136,Will,19,335


- Almacena el DF resultado anterior en una variable de Python denominada para poder hacer operaciones posteriores

In [57]:
personas_12_20 = people.select("*").filter(people['age'] >= 12).filter(people['age'] <= 20)

- Partiendo del resultado anterior, obtener el número de personas agrupado por edad, esto es, recuento de personas de la misma edad. Pista: Tendrás que usar la función groupBy('col') y la función count()

In [63]:
personas_12_20.groupBy('age').count()

age,count
20,5
19,11
18,8
